In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import logging
from model.MEFL import MEFARG
from dataset import *
from utils import *
from conf import get_config,set_logger,set_outdir,set_env
from PIL import Image
import cv2

/home/luqman/.local/share/virtualenvs/ME-GraphAU-NHAhuV8b/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DIR = '/home/luqman/images'
data = []
for f in os.listdir(DIR):
    img_path = os.path.join(DIR, f)
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    data.append(img)

In [3]:
data = np.array(data)
inputs = torch.tensor(data, dtype=torch.float32)
inputs = inputs.permute(0,3,1,2)
inputs.shape

torch.Size([5, 3, 224, 224])

In [4]:
net = MEFARG(num_classes=12, backbone="swin_transformer_base")
net_fold1 = load_state_dict(net, "/nas/project_data/B1_Behavior/hakim/ME-GraphAU/pretrained/swin/MEFARG_swin_base_BP4D_fold1.pth")
net_fold2 = load_state_dict(net, "/nas/project_data/B1_Behavior/hakim/ME-GraphAU/pretrained/swin/MEFARG_swin_base_BP4D_fold2.pth")
net_fold3 = load_state_dict(net, "/nas/project_data/B1_Behavior/hakim/ME-GraphAU/pretrained/swin/MEFARG_swin_base_BP4D_fold3.pth")

if torch.cuda.is_available():
    net_fold1 = nn.DataParallel(net_fold1).cuda()
    net_fold2 = nn.DataParallel(net_fold2).cuda()
    net_fold3 = nn.DataParallel(net_fold3).cuda()

/home/luqman/.local/share/virtualenvs/ME-GraphAU-NHAhuV8b/lib/python3.8/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
# input = torch.randn(1,3,224,224)

In [6]:
outputs_fold1, _ = net_fold1(inputs)
outputs_fold2, _ = net_fold2(inputs)
outputs_fold3, _ = net_fold3(inputs)

In [7]:
outputs = (outputs_fold1 + outputs_fold2 + outputs_fold3) / 3

In [8]:
outputs

tensor([[0.5154, 0.4615, 0.5323, 0.5475, 0.5435, 0.4841, 0.5294, 0.5528, 0.4867,
         0.5254, 0.4639, 0.4579],
        [0.4950, 0.5039, 0.4722, 0.5186, 0.5167, 0.4796, 0.5363, 0.5002, 0.4182,
         0.5214, 0.4967, 0.5010],
        [0.4759, 0.4312, 0.4867, 0.5492, 0.5154, 0.5741, 0.5011, 0.5314, 0.5342,
         0.5258, 0.4693, 0.4772],
        [0.5056, 0.4446, 0.5021, 0.5415, 0.5333, 0.5081, 0.5627, 0.5688, 0.5023,
         0.5673, 0.5538, 0.5687],
        [0.5213, 0.5294, 0.5058, 0.5513, 0.5483, 0.5427, 0.5039, 0.5225, 0.4966,
         0.5131, 0.4299, 0.4118]], device='cuda:0', grad_fn=<DivBackward0>)

# Labels

In [19]:
import pandas as pd

In [20]:
path_label = '/nas/project_data/B1_Behavior/hakim/ME-GraphAU/data/BP4D/AU_labels/F001_T1.csv'
usecols = ['0', '1', '2', '4', '6', '7', '10', '12', '14', '15', '17', '23', '24']
df = pd.read_csv(path_label, header=0, index_col=0, usecols=usecols)

In [21]:
df.head()

,1,2,4,6,7,10,12,14,15,17,23,24
0,,,,,,,,,,,,
2440,0,0,0,0,0,1,1,0,0,0,0,0
2441,0,0,0,0,0,1,1,0,0,0,0,0
2442,0,0,0,0,0,1,1,0,0,0,0,0
2443,0,0,0,0,0,1,1,0,0,0,0,0
2444,0,0,0,0,0,1,1,0,0,0,0,0
